In [ ]:
import ipywidgets as widgets
import math
import os
from IPython.display import display
from IPython.display import HTML
from IPython.display import clear_output
from IPython.display import FileLink

In [ ]:
# to create FloatText
floattext_data = {}
def floattext_maker(title_text, label_unit_list, explanation_HTML, key_prefix):
    title = widgets.HTML(f"<div style = 'font-size: 16px'><b>{title_text}</b></div>",
                        layout = widgets.Layout(margin = '0 0 10px 0')
                        )
    
    boxes = []
    key_prefix_placeholder = placeholder_dict.get(key_prefix, {})
    
    for label, unit in label_unit_list:
        key = f"{key_prefix}_{label}"
        
        placeholder = key_prefix_placeholder.get(label.lower(), '')
        text = widgets.Text(placeholder = placeholder,
                            description = label,
                            layout = widgets.Layout(width = 'auto', min_width = '100px',
                                                    max_width = '400px'),
                            style = {'description_width': 'auto'}
                           )
        error_msg = widgets.HTML(value='', layout=widgets.Layout(margin = '2px 0 0 0'))
        def validate_input(change, error = error_msg):
            try:
                if change['new'] == '':
                    error.value = ''
                else:
                    float(change['new'])
                    error.value = ''
            except ValueError:
                error.value = "<span style='color:red; font-size:12px'>Please enter a number.</span>"
        text.observe(validate_input, names='value')
        
        floattext_data[key] = text
        floattext_unit = widgets.HTML(value = unit,
                                      layout = widgets.Layout(margin = '0 0 0 10px'))
        box = widgets.HBox([text, floattext_unit])
        boxes.append(widgets.VBox([box, error_msg]))
    explanation = widgets.HTML(value = explanation_HTML,
                              layout = widgets.Layout(margin = '10px 0 0 0')
                              )
    return widgets.VBox([title] + boxes + [explanation],
                        layout = widgets.Layout(align_items = 'flex-start')
                       )

In [ ]:
# to create RadioButtons
_style = widgets.HTML(
        """
        <style>
        .widget-radio-box {
            display: flex !important;
            flex-direction: row !important;
            flex-wrap: nowrap !important;
            overflow: hidden !important;
            width: auto !important;
        }
        .widget-radio-box label {
            margin: 5px 10px 5px 0 !important;
            padding-left: 10px !important;
            white-space: nowrap !important;
        }
        </style>
        """,
        layout=widgets.Layout(display='none')
    )
radiobutton_data = {}
def radiobutton_maker(title_text, options_list, label_text, explanation_HTML, key):
    title = widgets.HTML(value = f"<div style = 'font-size: 16px'><b>{title_text}</b></div>",
                        layout = widgets.Layout(margin = '0 0 10px 0')
                        )
    radiobutton = widgets.RadioButtons(options = options_list,
                                       description = '',
                                       layout = widgets.Layout(width='auto'),
                                       style = {'description_width': '0px'}
                                )
    radiobutton_data[key] = radiobutton

    if explanation_HTML is not None:
        explanation = widgets.HTML(value = explanation_HTML,
                                layout = widgets.Layout(margin = '10px 0 0 0')
                                )
        explanation.layout.display = 'none'

        toggle_button = widgets.Button(
            description='▶ open explanation',
            layout=widgets.Layout(width='150px')
        )
        def toggle_explanation(btn):
            if explanation.layout.display == 'none':
                explanation.layout.display = 'block'
                btn.description = '▼ close explanation'
            else:
                explanation.layout.display = 'none'
                btn.description = '▶ open explanation'
        toggle_button.on_click(toggle_explanation)
        children_list = [title, radiobutton, toggle_button, explanation, _style]
    else:
        children_list = [title, radiobutton, _style]

    radiobutton_row = widgets.HBox([radiobutton],
        layout=widgets.Layout(
            display='flex',
            flex_flow='row',
            width='100%',
            min_width='800px',
            max_width='1000px'
        )
    )
    
    title_toggle_box = widgets.HBox([title, radiobutton],
                                    layout = widgets.Layout(align_items = 'flex-start'))
    
    box = widgets.VBox([title_toggle_box] + (children_list[2:] if explanation_HTML is not None else []),
                       layout = widgets.Layout(align_items = 'flex-start')
                      )
    return box

In [ ]:
# to show images
# to use: 
# name = image_presenter("pic_name.png", caption = 'caption', width = 200, height = 200)
# name = image_presenter("pic_name.png", width = 200, height = 200) - no caption
def image_presenter(image_path, caption = None, width = 500, height = 400):
    with open(image_path, 'rb') as f:
        # to extract the extension
        ext = os.path.splitext(image_path)[1][1:].lower()
        
        image = widgets.Image(value = f.read(), format = ext,
                                     width = width, height = height,
                                     layout = widgets.Layout(margin = '10px 0 0 0')
                                    )
        # to add caption
        if caption:
            caption_html = widgets.HTML(value = f"<div style = 'text-align: center'>{caption}</div>")
            image_widget = widgets.VBox([image, caption_html], 
                                         layout = widgets.Layout(width = f'{width}px',
                                                                 margin = '0 0 20px 0'
                                                                )
                                        )
        else:
            image_widget = widgets.VBox([image], 
                                        layout = widgets.Layout(width = f'{width}px',
                                                                margin = '0 0 20px 0'
                                                               )
                                       )
            return image_widget
            
    return image_widget

In [ ]:
# to extract FloatText values
# to use as a variable: {key_prefix}_values["label"], e.g. cwof_values["width"]
def get_floattext_values(key_prefix, label_unit_list):
    floattext_values = {}
    for label, unit in label_unit_list:
        try:
            widget_key = f"{key_prefix}_{label}"
            widget = floattext_data.get(widget_key)
            if widget_key in floattext_data and floattext_data[widget_key] is not None:
                raw_value = floattext_data[widget_key].value
                if raw_value and raw_value.strip() != '':
                    value = float(raw_value.strip())
                else:
                    value = None
            else:
                value = None
        except (ValueError, AttributeError):
            value = None
        floattext_values[label] = value
    return floattext_values

In [ ]:
# list of dimensions and unit
dimension_unit_list = [("weight", "[kg]"),
                       ("width", "[mm]"),
                       ("height", "[mm]"),
                       ("diameter", "[mm]"),
                       ("thickness", "[mm]"),
                       ("average thickness of painted canvas", "[mm]"),
                       ("number of partitions", ""),
                       ("width of one partition", "[mm]"),
                       ("height of one partition", "[mm]"),
                       ("average thickness of length and cross bars", "[mm]"),
                       ("width of glazing", "[mm]"),
                       ("height of glazing", "[mm]"),
                       ("diameter of glazing", "[mm]"),
                       ("thickness of glazing", "[mm]")
]

In [ ]:
# dictionaries for placeholder of each group (use with key_prefix)
placeholder_dict = {
    'canvas': {
        'weight': 'enter weight',
        'width': 'e.g. 1600',
        'height': 'e.g. 2000',
        'diameter': 'e.g. 1500',
        'average thickness of painted canvas': 'e.g. 3.9'
    },
    'stretcher': {
        'weight': 'enter weight',
        'width': 'e.g. 1600',
        'height': 'e.g. 2000',
        'diameter': 'e.g. 1500',
        'thickness': 'e.g. 200',
        'number of partitions': 'e.g. 4',
        'width of one partition': 'e.g. 500',
        'height of one partition': 'e.g. 700',
        'average thickness of length and cross bars': 'e.g. 200'
    },
    'cwf': {
        'weight': 'enter weight',
        'width': 'e.g. 2000',
        'height': 'e.g. 2400',
        'diameter': 'e.g. 2000',
        'thickness': 'e.g. 600',
        'width of glazing': 'e.g. 1800',
        'height of glazing': 'e.g. 2200',
        'diameter of glazing': 'e.g. 1700',
        'thickness of glazing': 'e.g. 5'
    },
    'canvasstretcher': {'weight': 'enter weight in total'},
    'csd': {'weight': 'enter weight in total'}
}

In [ ]:
# to explain how to use this app
preface_explanation = """
This application recommends a suitable backing board construction tailored to your artwork (easel paintings only).<br>
The maximum supported size is 150cm × 100cm (or 100cm × 150cm).<br>
For more accurate calculations and recommendations, <b>we strongly encourage you to fill in all input fields following the tabs from left to right.</b><br>
Please note that if some measurements are unavailable and certain fields are left blank, the accuracy of the results may be reduced.<br>
<b>Fields marked with a red box are mandatory — without these, the recommendation cannot be completed.</b><br>
<br>
Once all inputs have been completed, the "Results" tab will provide links to purchase the recommended materials,<br>
along with a downloadable PDF containing detailed instructions on how to attach them.<br>
After purchasing all the recommended materials, simply follow the steps in the PDF to complete your backing board construction.
"""
preface_box = widgets.HTML(preface_explanation)

# for preliminary
_mea_css = widgets.HTML("""
<style>
.force-vertical .widget-radio-box { 
    flex-direction: column !important; 
    align-items: flex-start !important;
}
.force-vertical .widget-radio-box label {
    display: block !important;
    margin: 4px 0 !important;
    white-space: normal !important;
}
</style>
""", layout=widgets.Layout(display='none')
)
measurement_HTML = widgets.HTML(
    """<div style = 'font-size: 16px;'><b>Measurements of artwork</b></div>
    We need measurements of each part of your artwork for recommendation (e.g. weight, size, thickness of the canvas, stretcher, and decoreative frame).<br>
    If you do not provide measurements for each part, some parts will be calculated using standard values.<br>
    This may reduce accuracy - for the most accurate result, we recommend <b>entering as many measurements as possible for each part</b>.<br>
    Please select how you are able to measure your artwork:
    """)
mea_radiobutton = widgets.RadioButtons(options = ['individually - canvas, stretcher, and decorative frame measured separately',
                                                  'partially - canvas + stretcher together, decorative frame separately',
                                                  'as a whole - canvas + stretcher + decorative frame together'],
                                       value = None,
                                       layout = {'width': 'max-content'}
                                      )
mea_radiobutton.add_class('force-vertical')
measurement_radiobutton = widgets.HBox([widgets.Label(value = 'I can measure the artwork'),
                                        _mea_css,
                                        mea_radiobutton])
measurement_box = widgets.VBox([measurement_HTML, measurement_radiobutton])
load_radiobutton = radiobutton_maker(
    "Planned mechanical load", ["transportation (loan)", "exhibition", "transportation and exhibition", "restoration treatment"],
    "planned mechanical loed", None, "load"
)
canvas_radiobutton = radiobutton_maker(
    "Format of the canvas", ["quadrangle", "oval", "circular"],
    "format of the canvas", None, "canvas"
)
preliminary_box = widgets.VBox([load_radiobutton, canvas_radiobutton, _style])

In [ ]:
# to show example images along to the shape
def show_shape_images(shape):
    for name, img in canvas_images.items():
        img.layout.display = 'flex' if name == shape else 'none'
    for name, img in stretcher_images.items():
        img.layout.display = 'flex' if name == shape else 'none'
    for name, img in cwf_images.items():
        img.layout.display = 'flex' if name == shape else 'none'
canvas_image1 = image_presenter("canvas-quadrangle-dimensions.png", caption = 'quadrangle',
                                width = 500, height = 400
                               )
canvas_image2 = image_presenter("canvas-oval-dimensions.png", caption = 'oval',
                                width = 500, height = 400
                               )
canvas_image3 = image_presenter("canvas-circular-dimensions.png", caption = 'circular',
                                width = 500, height = 400
                               )
canvas_images = {
    "quadrangle": canvas_image1,
    "oval": canvas_image2,
    "circular": canvas_image3,
}
canvas_image_box = widgets.HBox([canvas_image1, canvas_image2, canvas_image3],
                               layout = widgets.Layout(display = 'flex', align_items = 'flex-end')
                               )
stretcher_image1 = image_presenter("stretcher-quadrangle-dimensions.jpg", caption = "quadrangle",
                                   width = 560, height = 400
                                   )
stretcher_image2 = image_presenter("stretcher-oval-dimensions.jpg", caption = "oval",
                                   width = 500, height = 400
                                   )
stretcher_image3 = image_presenter("stretcher-circular-dimensions.jpg", caption = "circular",
                                   width = 560, height = 400
                                   )
stretcher_images = {
    "quadrangle": stretcher_image1,
    "oval": stretcher_image2,
    "circular": stretcher_image3,
}
stretcher_image_box = widgets.Box([stretcher_image1, stretcher_image2, stretcher_image3],
                                  layout = widgets.Layout(display = 'flex', align_items = 'flex-end')
                                  )
cwf_image1 = image_presenter("outer_frame-quadrangle-dimensions.png", caption = 'quadrangle',
                             width = 500, height = 400
                             )
cwf_image2 = image_presenter("outer_frame-oval-dimensions.png", caption = 'oval',
                             width = 500, height = 400
                             )
cwf_image3 = image_presenter("outer_frame-circular-dimensions.png", caption = 'circular',
                             width = 500, height = 400
                             )
cwf_images = {
    "quadrangle": cwf_image1,
    "oval": cwf_image2,
    "circular": cwf_image3
}
cwf_image_box = widgets.HBox([cwf_image1, cwf_image2, cwf_image3],
                             layout = widgets.Layout(display = 'flex', align_items = 'flex-end')
                             )
canvas_stretcher_image_box = widgets.HBox([canvas_image_box, stretcher_image_box])
csd_image_box = widgets.HBox([canvas_image_box, stretcher_image_box, cwf_image_box])

# to extract units from dimension_unit_list
def pick_units(*labels):
    return [u for u in dimension_unit_list if u[0] in labels]

# 1. individually
# dimensions of canvas without decorative frame(key = "canvas")
canvas_HTML = widgets.HTML(
    """<div style = 'font-size: 16px;'><b>Tips before you begin</b></div>
    On this tab, enter <b>canvas-only</b> values - for example, the weight should be the canvas weight.<br>
    If you cannot measure the canvas on its own, please return to the 'Preliminary tab' and change the option under 'Measurements of artwork'.<br><br>
    """)
canvas_explanation = """
<small>
<b>For improved accuracy, entering the measured thickness is recommended.</b><br>
If the thickness of painting is not provided, the base thickness is estimated based on the presence and number of lining layers.<br>
Without lining (0 layers), it is set to 2–2.5 mm.<br>
With lining, the base thickness is set to 2–3 times the value with lining.<br>
</small>
"""
canvas_float_box_nor = floattext_maker("Canvas",
                                       pick_units("weight", "width", "height", "average thickness of painted canvas"),
                                       canvas_explanation, "canvas"
                                      )
canvas_nor = {
    "weight": floattext_data.get("canvas_weight"),
    "average thickness of painted canvas": floattext_data.get("canvas_average thickness of painted canvas"),
    "width": floattext_data.get("canvas_width"),
    "height": floattext_data.get("canvas_height")
}                                      
canvas_float_box_cir = floattext_maker("Canvas",
                                       pick_units("weight", "diameter", "average thickness of painted canvas"),
                                       canvas_explanation, "canvas"
                                      )
canvas_cir = {
    "weight": floattext_data.get("canvas_weight"),
    "average thickness of painted canvas": floattext_data.get("canvas_average thickness of painted canvas"),
    "diameter": floattext_data.get("canvas_diameter")
}
canvas_float_box_cir.layout.display = 'none'
canvas_float_boxes = widgets.VBox([canvas_float_box_nor, canvas_float_box_cir])
canvas_box = widgets.VBox([canvas_HTML, canvas_float_boxes, canvas_image_box, _style])

# dimensions of stretcher(key = "stretcher")
stretcher_HTML = widgets.HTML(
    """<div style = 'font-size: 16px;'><b>Tips before you begin</b></div>
    On this tab, enter <b>stretcher-only</b> values - for example, the weight should be the stretcher weight.<br>
    If you cannot measure the canvas on its own, please return to the 'Preliminary tab' and change the option under 'Measurements of artwork'.<br><br>
    """)
stretcher_explanation = """
<small>
description needed
</small>
"""
stretcher_radiobutton = radiobutton_maker("Type of wood", ["soft wood", "hard wood"],
                                         "type of wood", None, "stretcher"
                                         )
stretcher_float_box_nor = floattext_maker("Stretcher",
                                       pick_units("weight", "width", "height", "thickness", "number of partitions", "width of one partition", "height of one partition", "average thickness of length and cross bars"),
                                       None, "stretcher"
                                      )
stretcher_nor = {
    "weight": floattext_data.get("stretcher_weight"),
    "thickness": floattext_data.get("stretcher_thickness"),
    "number of partitions": floattext_data.get("stretcher_number of partitions"),
    "width of one partition": floattext_data.get("stretcher_width of one partition"),
    "height of one partition": floattext_data.get("stretcher_height of one partition"),
    "average thickness of length and cross bars": floattext_data.get("stretcher_average thickness of length and cross bars"),
    "width": floattext_data.get("stretcher_width"),
    "height": floattext_data.get("stretcher_height")
}
stretcher_float_box_cir = floattext_maker("Stretcher",
                                       pick_units("weight", "diameter", "thickness", "number of partitions", "width of one partition", "height of one partition", "average thickness of length and cross bars"),
                                       None, "stretcher"
                                      )
stretcher_cir = {
    "weight": floattext_data.get("stretcher_weight"),
    "thickness": floattext_data.get("stretcher_thickness"),
    "number of partitions": floattext_data.get("stretcher_number of partitions"),
    "width of one partition": floattext_data.get("stretcher_width of one partition"),
    "height of one partition": floattext_data.get("stretcher_height of one partition"),
    "average thickness of length and cross bars": floattext_data.get("stretcher_average thickness of length and cross bars"),
    "diameter": floattext_data.get("stretcher_diameter")
}                                      
stretcher_float_box_cir.layout.display = 'none'
stretcher_float_boxes = widgets.VBox([stretcher_float_box_nor, stretcher_float_box_cir])
stretcher_box = widgets.VBox([stretcher_HTML, stretcher_float_boxes, stretcher_radiobutton, stretcher_image_box, _style])

# dimensions of canvas with decorative frame(key = "cwf")
cwf_HTML = widgets.HTML(
    """<div style = 'font-size: 16px;'><b>Tips before you begin</b></div>
    On this tab, enter <b>decorative frame-only</b> values - for example, the weight should be the decorative frame weight.<br>
    If you cannot measure the canvas on its own, please return to the 'Preliminary tab' and change the option under 'Measurements of artwork'.<br>
    <b>Glazing</b> refers to the kind of glass the glazing is made of and the measurements taken for <b>the glazing only</b>.<br><br>
    """)
cwf_explanation = """
<small>
description needed
</small>
"""
cwf_radiobutton = radiobutton_maker("Glazing", ["weight with glazing", "weight without glazing"],
                                    "glazing", None, "cwf"
                                    )
radiobutton_data["cwf"].value = None
glazing_radiobutton = radiobutton_maker("Type of glazing", ["plain glass", "museums glass", "acrylic glass"],
                                        "type of glazing", None, "glazing")
radiobutton_data["glazing"].value = None
glazing_radiobutton.layout.display = 'none'
cwf_float_box_nor = floattext_maker("Decorative frame",
                                    pick_units("weight", "width", "height", "thickness"),
                                    None, "cwf"
                                    )
glazing_float_box_nor = floattext_maker("Dimensions of glazing",
                                        pick_units("width of glazing", "height of glazing", "thickness of glazing"),
                                        None, "cwf"
                                       )
glazing_float_box_nor.layout.display = 'none'
cwf_nor = {
    "weight": floattext_data.get("cwf_weight"),
    "thickness": floattext_data.get("cwf_thickness"),
    "width": floattext_data.get("cwf_width"),
    "height": floattext_data.get("cwf_height"),
    "width of glazing": floattext_data.get("cwf_width of glazing"),
    "height of glazing": floattext_data.get("cwf_height of glazing"),
    "thickness of glazing": floattext_data.get("cwf_thickness of glazing")
}                                    
cwf_float_box_cir = floattext_maker("Decorative frame",
                                    pick_units("weight", "diameter", "thickness"),
                                    None, "cwf"
                                    )
glazing_float_box_cir = floattext_maker("Dimensions of glazing",
                                        pick_units("diameter of glazing", "thickness of glazing"),
                                        None, "cwf"
                                       )
glazing_float_box_cir.layout.display = 'none'
cwf_cir = {
    "weight": floattext_data.get("cwf_weight"),
    "thickness": floattext_data.get("cwf_thickness"),
    "diameter": floattext_data.get("cwf_diameter"),
    "diameter of glazing": floattext_data.get("cwf_diameter of glazing"),
    "thickness of glazing": floattext_data.get("cwf_thickness of glazing")
}                                    
cwf_float_box_cir.layout.display = 'none'
cwf_float_boxes = widgets.VBox([cwf_float_box_nor, cwf_float_box_cir, cwf_radiobutton])
glazing_float_boxes = widgets.VBox([glazing_radiobutton, glazing_float_box_nor, glazing_float_box_cir])
cwf_box = widgets.VBox([cwf_HTML, cwf_float_boxes, glazing_float_boxes, cwf_image_box])

# 2. partially
# dimensions of canvas + stretcher - name: canvas_stretcher, key: canvasstretcher
canvas_stretcher_HTML = widgets.HTML(
    """<div style = 'font-size: 16px;'><b>Tips before you begin</b></div>
    You have chosen partial measurement, but please provide as much detail as possible for more accurate results.<br>
    If there are parameters you cannot measure, you may simply skip them.<br>
    The following four sections will appear: Total, Canvas, Stretcher, and Type of wood.<br>
    <b>Total</b> refers to the measurements taken for both <b>the canvas and the stretcher together</b>.<br>
    <b>Canvas</b> refers to the measurements taken for <b>the canvas only</b>.<br>
    <b>Stretcher</b> refers to the measurements taken for <b>the stretcher only</b>.<br>
    <b>Type of wood</b> refers to the kind of wood the stretcher is made of.<br><br>
    """)
canvas_stretcher_float_nor = floattext_maker("Total (canvas + stretcher)",
                                              pick_units("weight"),
                                              None, "canvasstretcher"
                                              )
canvas_stretcher_float_box_nor = widgets.VBox([canvas_stretcher_float_nor, canvas_float_box_nor, stretcher_float_box_nor])
canvas_stretcher_nor = {
    "weight": floattext_data.get("canvasstretcher_weight")
}
canvas_stretcher_float_cir = floattext_maker("Total (canvas + stretcher)",
                                             pick_units("weight"),
                                             None, "canvasstretcher"
                                             )
canvas_stretcher_cir = {
    "weight": floattext_data.get("canvasstretcher_weight")
}
canvas_stretcher_float_box_cir = widgets.VBox([canvas_stretcher_float_cir, canvas_float_box_cir, stretcher_float_box_cir])
canvas_stretcher_float_box_cir.layout.display = 'none'
canvas_stretcher_float_boxes = widgets.VBox([canvas_stretcher_float_box_nor, canvas_stretcher_float_box_cir])
canvas_stretcher_box = widgets.VBox([canvas_stretcher_HTML, canvas_stretcher_float_boxes, stretcher_radiobutton, canvas_stretcher_image_box, _style])

# 3. as a whole artwork
# dimensions of a whole artwork (canvas + stretcher + decorative frame) - name: csd, key: csd
csd_HTML = widgets.HTML(
    """<div style = 'font-size: 16px;'><b>Tips before you begin</b></div>
    You have chosen measurement that a whole artwork measured together, but please provide as much detail as possible for more accurate results.<br>
    If there are parameters you cannot measure, you may simply skip them.<br>
    The following six sections will appear: Total, Canvas, Stretcher, Type of wood, Decorative frame and Glazing.<br>
    <b>Total</b> refers to the measurements taken for both <b>the canvas, the stretcher, and the decorative frame together</b>.<br>
    <b>Canvas</b> refers to the measurements taken for <b>the canvas only</b>.<br>
    <b>Stretcher</b> refers to the measurements taken for <b>the stretcher only</b>.<br>
    <b>Type of wood</b> refers to the kind of wood the stretcher is made of.<br>
    <b>Decorative frame</b> refers to the measurements taken for <b>the decorative frame only</b>.<br>
    <b>Glazing</b> refers to the kind of glass the glazing is made of and the measurements taken for <b>the glazing only</b>.<br><br>
    """)
csd_float_nor = floattext_maker("Total (canvas + stretcher + decorative frame)",
                                pick_units("weight"),
                                None, "csd"
                                )
csd_float_box_nor = widgets.VBox([csd_float_nor, canvas_float_box_nor, stretcher_float_box_nor, stretcher_radiobutton, cwf_float_box_nor, cwf_radiobutton, glazing_radiobutton, glazing_float_box_nor])
csd_nor = {
    "weight": floattext_data.get("csd_weight")
}
csd_float_cir = floattext_maker("Total (canvas + stretcher + decorative frame)",
                                pick_units("weight"),
                                None, "csd"
                                )
csd_float_box_cir = widgets.VBox([csd_float_cir, canvas_float_box_cir, stretcher_float_box_cir, stretcher_radiobutton, cwf_float_box_cir, cwf_radiobutton, glazing_radiobutton, glazing_float_box_cir])
csd_cir = {
    "weight": floattext_data.get("csd_weight")
}
csd_float_box_cir.layout.display = 'none'
csd_float_boxes = widgets.VBox([csd_float_box_nor, csd_float_box_cir])
csd_box = widgets.VBox([csd_HTML, csd_float_boxes, csd_image_box, _style])

In [ ]:
def _toggle_individual_weights(hide: bool):
    mea_mode = (str(mea_radiobutton.value) if mea_radiobutton.value is not None else "").lower()
    if mea_mode.startswith("as a whole"):
        keys = ["canvas_weight", "stretcher_weight", "cwf_weight"]
    elif mea_mode.startswith("partially"):
        keys = ["canvas_weight", "stretcher_weight"]
    else:
        keys = ["canvas_weight", "stretcher_weight", "cwf_weight"]
        hide = False
    for key in keys:
        w = floattext_data.get(key)
        if w:
            w.layout.display = 'none' if hide else 'flex'
        
def _rebind_shared(is_circle: bool):
    # canvas
    src = canvas_cir if is_circle else canvas_nor
    for k in ("weight", "average thickness of painted canvas", "width", "height", "diameter"):
        if k in src and src[k] is not None:
            floattext_data[f"canvas_{k}"] = src[k]
    # stretcher
    src = stretcher_cir if is_circle else stretcher_nor
    for k in ("weight", "thickness", "number of partitions",
              "width of one partition", "height of one partition",
              "average thickness of length and cross bars",
              "width", "height", "diameter"):
        if k in src and src[k] is not None:
            floattext_data[f"stretcher_{k}"] = src[k]
    # cwf
    src = cwf_cir if is_circle else cwf_nor
    for k in ("weight", "thickness", "width", "height", "diameter",
              "width of glazing", "height of glazing", "diameter of glazing", "thickness of glazing"):
        if k in src and src[k] is not None:
            floattext_data[f"cwf_{k}"] = src[k]
    # canvas + stretcher
    canvas_stretcher_data = canvas_stretcher_cir if is_circle else canvas_stretcher_nor
    canvas_data = canvas_cir if is_circle else canvas_nor
    stretcher_data = stretcher_cir if is_circle else stretcher_nor
    w = canvas_stretcher_data.get("weight")
    if w is not None:
        floattext_data["canvasstretcher_weight"] = w
    for k, v in canvas_data.items():
        if v is not None:
            floattext_data[f"canvas_{k}"] = v
    for k, v in stretcher_data.items():
        if v is not None:
            floattext_data[f"stretcher_{k}"] = v
    # csd
    csd_data = csd_cir if is_circle else csd_nor
    canvas_data = canvas_cir if is_circle else canvas_nor
    stretcher_data = stretcher_cir if is_circle else stretcher_nor
    cwf_data = cwf_cir if is_circle else cwf_nor
    w = csd_data.get("weight")
    if w is not None:
        floattext_data["csd_weight"] = w
    for k, v in canvas_data.items():
        if v is not None:
            floattext_data[f"canvas_{k}"] = v
    for k, v in stretcher_data.items():
        if v is not None:
            floattext_data[f"stretcher_{k}"] = v
    for k, v in cwf_data.items():
        if v is not None:
            floattext_data[f"cwf_{k}"] = v

def active_units(section: str):
    shape = radiobutton_data["canvas"].value
    normal_labels = {
        "canvas": ["weight", "width","height", "average thickness of painted canvas"],
        "stretcher": ["weight", "width","height", "thickness","number of partitions",
                      "width of one partition", "height of one partition",
                      "average thickness of length and cross bars"],
        "cwf": ["weight", "width", "height", "thickness", "width of glazing", "height of glazing", "thickness of glazing"],
        "canvasstretcher": ["weight"],
        "csd": ["weight"]
    }
    circular_labels = {
        "canvas": ["weight", "diameter", "average thickness of painted canvas"],
        "stretcher": ["weight", "diameter", "thickness","number of partitions",
                      "width of one partition", "height of one partition",
                      "average thickness of length and cross bars"],
        "cwf": ["weight", "diameter", "thickness", "diameter of glazing", "thickness of glazing"],
        "canvasstretcher": ["weight"],
        "csd": ["weight"]
    }
    labels = circular_labels[section] if shape == "circular" else normal_labels[section]
    unit_by_label = dict(dimension_unit_list)
    unit_pairs = [(lbl, unit_by_label[lbl]) for lbl in labels if lbl in unit_by_label]
    unit_map   = {lbl: unit_by_label[lbl] for lbl in labels if lbl in unit_by_label}
    return unit_pairs, unit_map

def on_canvas_shape(change):
    shape = change["new"]
    is_circle = (shape == "circular")
    canvas_float_box_nor.layout.display = 'none' if is_circle else 'flex'
    canvas_float_box_cir.layout.display = 'flex' if is_circle else 'none'
    stretcher_float_box_nor.layout.display = 'none' if is_circle else 'flex'
    stretcher_float_box_cir.layout.display = 'flex' if is_circle else 'none'
    cwf_float_box_nor.layout.display = 'none' if is_circle else 'flex'
    cwf_float_box_cir.layout.display = 'flex' if is_circle else 'none'
    cwf_glazing_value = radiobutton_data["cwf"].value
    cwf_glazing_value = radiobutton_data["cwf"].value
    if cwf_glazing_value == "weight with glazing":
        glazing_float_box_nor.layout.display = 'none' if is_circle else 'block'
        glazing_float_box_cir.layout.display = 'block' if is_circle else 'none'
    else:
        glazing_float_box_nor.layout.display = 'none'
        glazing_float_box_cir.layout.display = 'none'
    canvas_stretcher_float_box_nor.layout.display = 'none' if is_circle else 'flex'
    canvas_stretcher_float_box_cir.layout.display = 'flex' if is_circle else 'none'
    csd_float_box_nor.layout.display = 'none' if is_circle else 'flex'
    csd_float_box_cir.layout.display = 'flex' if is_circle else 'none'
    
    _rebind_shared(is_circle)
    mea_mode = (str(mea_radiobutton.value) if mea_radiobutton.value is not None else "").lower()
    hide = mea_mode.startswith("partially") or mea_mode.startswith("as a whole")
    _toggle_individual_weights(hide)
     
    to_clear = ["width", "height", "width of glazing", "height of glazing"] if is_circle else ["diameter", "diameter of glazing"]
    for label in to_clear:
        w = floattext_data.get(f"canvas_{label}")
        if w: w.remove_class("red-input")
        w = floattext_data.get(f"stretcher_{label}")
        if w: w.remove_class("red-input")
        w = floattext_data.get(f"cwf_{label}")
        if w: w.remove_class("red-input")
    show_shape_images(shape)
    if 'f_calculation' in globals():
        f_calculation() 

radiobutton_data["canvas"].observe(on_canvas_shape, names='value')
on_canvas_shape({"new": radiobutton_data["canvas"].value})

In [ ]:
# toggle to choose - with yes(frequency known) or no(frequency unknown)
f_toggle_title = widgets.HTML(
    value = f"<div style = 'font-size: 16px;'><b>I know the first natural frequency: </b></div>"
)
f_toggle = widgets.ToggleButtons(options = [('yes', True), ('no', False)],
                                 description = '', disabled = False
                                )
f_toggle_box = widgets.VBox([f_toggle_title, f_toggle])

# if yes, enter the known first natural frequency
f_known = widgets.FloatText(description = 'first natural frequency:',
                            layout = widgets.Layout(width = 'auto',
                                                             min_width = '100px',
                                                             max_width = '200px'),
                            style = {'description_width': 'auto'},
                           )
f_known_unit = widgets.Label('[Hz]',
                       layout = widgets.Layout(margin = '0 0 0 10px')
                            )
                            
f_known_box = widgets.HBox([f_known, f_known_unit])

In [ ]:
# if no, go to calculation part
f_unknown_guidance = widgets.HTML("""<div style = 'margin: 20px 0 0 0'>
The first natural frequency will be calculated based on the following parameters:
</div>
""")

# but first, enter following parameters - a) pretension
pretension_explanation = """
<small>
<b>how much is the canvas stretched?</b>
<ul style = "padding-left: 15px; list-style-type: disc;">
    <li>low:
    <ul style = "padding-left: 15px">
        <li>Canvas is limp</li>
        <li>Surface moves easily visible while painting is moved even slightly</li>
        <li>Surface sags visibly or forms a deformation at lower edge of canvas</li>
        <li>When touching it, you can clearly feel lots of space for movement 
            and/or canvas can be easily moved</li>
    </ul></li>
    <li>low-medium:
    <ul style = "padding-left: 15px">
        <li>Canvas is stretched irregularly</li>
        <li>Canvas has both areas that can be assigned to low and medium tension</li>
    </ul></li>
    <li>medium (ideal):
    <ul style = "padding-left: 15px">
        <li>Ideal pretension for a canvas painting</li>
        <li>Tension is low enough to allow necessary evasive movements during handling/
            transport, but still high enough to protect the paint layers from stronger deformations</li>
        <li>The movement during handling/transport is only slightly visible</li>
        <li>When touching it, there is a little space for movement if necessary, 
            but it is sufficiently under tension for carrying the weight of the paint 
            layers and forms a flat canvas surface</li>
    </ul></li>
    <li>medium-high:
    <ul style = "padding-left: 15px">
        <li>Canvas is stretched irregularly</li>
        <li>Canvas has  both areas that can be assigned to medium and high tension</li>
    </ul></li>
    <li>high:
    <ul style = "padding-left: 15px">
        <li>Canvas pretension is too high for painting</li>
        <li>Canvas is stretched unnecessarily far and will lead to irreversible 
            overstretching/wearing of the canvas over a long period of time</li>
        <li>When touched, canvas surface will feel like a drum</li>
    </ul></li>
</ul>
</small>
"""
pretension_box = radiobutton_maker(
    "pretension", ["low", "low-medium", "medium (ideal)", "medium-high", "high"],
    "pretension", pretension_explanation, "pretension"
)

# b) lining
lining_explanation = """
<small>
<b>how many lining layers are on the painting?</b>
<ul style = "padding-left: 15px">
    <li>0: The painting is not lined - there is no lining on the painting</li>
    <li>1: one additional canvas</li>
    <li>2: two additional canvasses</li>
    <li>3: three additional canvasses</li>
</ul>
</small>
"""
lining_box = radiobutton_maker(
    "lining", ["0", "1", "2", "3"],
    "lining", lining_explanation, "lining"
)

# c) deformation
deformation_explanation = """
<small>
<b>How much is the canvas deformed?</b>
<ul style = "padding-left: 15px">
    <li>not at all: the backside of the canvas is flat</li>
    <li>medium: the backside of the canvas is slightly deformed</li>
    <li>heavy: the backside of the canvas is strongly deformed</li>
</ul>
</small>
"""
deformation_box = radiobutton_maker(
    "deformation", ["not at all", "medium", "heavy"],
    "deformation", deformation_explanation, "deformation"
)

# d) impasto-painted surface
impasto_explanation = """
<small>
<b>how much surfaces are impasto-painted?</b>
<ul style = "padding-left: 15px">
    <li>0%: there is no impasto-painted surface on canvas</li>
    <li>0%: there is no impasto-painted surface on canvas</li>
    <li>10%: Approximately 10% of the painted canvas is applied in a pasty manner</li>
    <li>20%: Approximately 20% of the painted canvas is applied in a pasty manner</li>
    <li>50%: Approximately 50% of the painted canvas is applied in a pasty manner</li>
</ul>
</small>
"""
impasto_box = radiobutton_maker(
    "impasto-painted surface", ["0%", "10%", "20%", "50%"],
    "impasto-painted surface", impasto_explanation, "impasto"
)

f_unknown_box = widgets.VBox([f_unknown_guidance, pretension_box,
                              lining_box, deformation_box,impasto_box]
                            )

In [ ]:
## 3. output area
# calculation output
cal_output = widgets.Output()
# warning output
warning_output = widgets.Output()
# display output
decision_output = widgets.Output()
# result output
result_output = widgets.Output()

In [ ]:
# make instruction download button
instruction_label = widgets.HTML("For installation instruction (click to download):&nbsp;")
instruction_link = widgets.Output()
with instruction_link:
    display(FileLink("instruction.pdf"))
download_box = widgets.HBox([instruction_label, instruction_link])

# disclaimer
disclaimer_HTML = widgets.HTML("""
Our application provides recommendations for backing board construction based on user-input values and engineering calculations.<br>
The application of these recommendations is solely at the discretion of the user, and we do not assume any legal liability.<br>
For more precise design guidance, please refer to the cited articles below.
<br><br>
<blockquote style = 'margin: 0 0 0 30px'>
    Kracht, K., Bisschoff, M., Leeuwestein:
    <i>Optimization of the Protection of the Kröller-Müller Museum’s wax-resin lined Van Gogh paintings from Shocks and Vibration.</i>
    Preprints of ICOM-CC Triennial Conference, Valencia, 2023.<br>
    <a href="https://www.icom-cc-publications-online.org/5683/Optimising-the-protection-of-the-Kroller-Muller-Museums-wax-resin-lined-van-Gogh-paintings-from-shocks-and-vibrations-in-transit"
        target="_blank" 
        style="color:blue; text-decoration:underline;">
        https://www.icom-cc-publications-online.org/5683/Optimising-the-protection-of-the-Kroller-Muller-Museums-wax-resin-lined-van-Gogh-paintings-from-shocks-and-vibrations-in-transit
    </a>
    <br><br>
    Kracht, K. van Oudheusden, S.:
    <i>Van Gogh in motion. Safeguarding lined and unlined Van Gogh paintings from vibration and mechanical shock during transport.</i>
    Prensentation and postprints of AIC's 53rd annual meeting, Minneapolis, 2025.<br>
    <a href="https://aic53rdannualmeeting2025.sched.com/event/1t4Lm/preventive-care-van-gogh-in-motion-safeguarding-lined-and-unlined-van-gogh-paintings-from-vibration-and-mechanical-shock-during-transport-remote-presentation"
        target="_blank" 
        style="color:blue; text-decoration:underline;">
        https://aic53rdannualmeeting2025.sched.com/event/1t4Lm/preventive-care-van-gogh-in-motion-safeguarding-lined-and-unlined-van-gogh-paintings-from-vibration-and-mechanical-shock-during-transport-remote-presentation
    </a>
    <br><br>
    Kracht, K., Hedinger, D., Radermacher, K..:
    <i>Vibrierende Kunst. Wie restauratorische Maßnahmen das Schwingungsverhalten von Beckmanns Fastnacht beeinflussen.</i>
    Restauro No. 7/2018, S. 45-49.
</blockquote>
""")
article_label = widgets.HTML("<div style = 'margin: 0 0 0 30px;'>Download (click to download):&nbsp;&nbsp;</div>")
article_en = widgets.Output()
article_de = widgets.Output()
with article_en:
    display(FileLink("English.pdf"))
sep = widgets.HTML("&nbsp;|&nbsp;")
with article_de:
    display(FileLink("Deutsch.pdf"))
article_box = widgets.HBox([article_label, article_en, sep, article_de])
credit_HTML = widgets.HTML("<div style = 'margin: 30px 0 0 0;'>© 2025 Technische Universität Hamburg, Norbert Hoffmann, Kerstin Annette Kracht, Jakob Ohlsen, Franziska Lipp, Hyeonjeong Kim</div>")
disclaimer_box = widgets.VBox([disclaimer_HTML, article_box, credit_HTML])

In [ ]:
# tabs of input boxes for dimension
tabs_titles = ["Preface", "Preliminary", "Canvas size", "Stretcher construction", "Decorative frame", "Results", "Disclaimer"]
tabs_children = [preface_box, widgets.VBox([preliminary_box, measurement_box, f_toggle_box, decision_output]), widgets.VBox([canvas_box, warning_output]), stretcher_box, cwf_box, widgets.VBox([result_output]), disclaimer_box]
tabs = widgets.Tab(children = tabs_children)
for i in range(len(tabs_titles)):
    tabs.set_title(i, tabs_titles[i])

_preliminary_box = tabs.children[1]
_results_box     = tabs.children[5]
_disclaimer_box  = tabs.children[6]

canvas_stretcher_tab = widgets.VBox([canvas_stretcher_box])
csd_tab = widgets.VBox([csd_box])

# for initial display
def set_initial_tabs():
    tabs.children = [preface_box, _preliminary_box, _results_box, _disclaimer_box]
    for i, title in enumerate(["Preface", "Preliminary", "Results", "Disclaimer"]):
        tabs.set_title(i, title)

# along to the selection of measurements of artwork
def apply_measurement_mode(mea_mode: str):
    if mea_mode == "individually":
        children = [preface_box, _preliminary_box, canvas_box, stretcher_box, cwf_box, _results_box, _disclaimer_box]
        titles   = ["Preface", "Preliminary", "Canvas size", "Stretcher construction", "Decorative frame", "Results", "Disclaimer"]
    elif mea_mode == "partially":
        children = [preface_box, _preliminary_box, canvas_stretcher_tab, cwf_box, _results_box, _disclaimer_box]
        titles   = ["Preface", "Preliminary", "Canvas + Stretcher", "Decorative frame", "Results", "Disclaimer"]
    elif mea_mode == "as a whole":
        children = [preface_box, _preliminary_box, csd_tab, _results_box, _disclaimer_box]
        titles   = ["Preface", "Preliminary", "A whole artwork", "Results", "Disclaimer"]
    else:
        set_initial_tabs()
        return
    tabs.children = children
    for i, t in enumerate(titles):
        tabs.set_title(i, t)
set_initial_tabs()
tabs.layout.width = "100%"

# summarize results and show the result tab
result_state = {"f": None, "mode": None, "mea_mode": None}
def show_group(title, items: dict, unit_map: dict = None):
    if all(v is None for v in items.values()):
        return
    html = f"<div style = 'margin: 0 0 0 15px;'>{title}<ul style = 'margin: 0 0 0 5px;'>"
    for key, value in items.items():
        if value is not None:
            unit = unit_map.get(key, "") if unit_map else ""
            html += f"<li>{key.replace('_', ' ')}: {value} {unit}</li>"
    html += "</ul></div>"
    display(widgets.HTML(html))

def result_summary(f = None, mode = None, mea_mode = None, inputs_only = False):
    global preliminary_values
    with result_output:
        clear_output()
        # show frequency
        if f is not None:
            label = "entered" if mode == 'entered' else "calculated"
            display(widgets.HTML(f"""
            <div style = 'margin: 0 0 10px 0;'>
            The {label} respectively measured first natural frequency of the painted canvas is {f:.1f} Hz.
            </div>"""
            ))
        else:
            display(widgets.HTML("No frequency entered/calculated yet."))
       
        # to show recommendations
        # common values for later recommendation
        canvas_shape = radiobutton_data["canvas"].value
        canvas_unit_pairs, _ = active_units("canvas")
        canvas_values = get_floattext_values("canvas", canvas_unit_pairs)
        load_value = radiobutton_data["load"].value
        pretension_value = radiobutton_data["pretension"].value
        lining_value = radiobutton_data["lining"].value
        deformation_value = radiobutton_data["deformation"].value
        impasto_value = radiobutton_data["impasto"].value

        # fiberfilling links
        WLG035_link = "https://caruso-ebersdorf.de/en/"
        WLG040_link = "https://caruso-ebersdorf.de/en/"
        WLG045_link = "https://caruso-ebersdorf.de/en/"

        # backboard panel links
        EBB8_link = "https://www.klug-conservation.com/corrugated-boards-ebb-8-0-mm?"
        Honey7_link = "https://www.klug-conservation.com/honeycomb-panels-079-natural-white-with-wave-structure?"
        Honey13_link = "https://www.klug-conservation.com/honeycomb-panels-071-natural-white?"
        
        # explanation for fiberfilling and backingboard panel
        fiberfilling_HTML = widgets.HTML(value = "The recommended fiberfilling material:")
        WLG035_HTML = widgets.HTML(value = f"""
        <div style = 'margin: 0 0 0 20px;'><li><b>Caruso Isobond WLG035</b> &nbsp; more information: <a href="{WLG035_link}"
        target="_blank" style="color:blue; text-decoration:underline; font-weight:bold;">Click here</a><br>
        The recommended backingboard construction is only applicable for <i>restoreation treatment</i>.</li></div>
        """)
        WLG040_HTML = widgets.HTML(value = f"""
        <div style = 'margin: 0 0 0 20px;'><li><b>Caruso Isobond WLG040</b> &nbsp; more information: <a href="{WLG040_link}"
        target="_blank" style="color:blue; text-decoration:underline; font-weight:bold;">Click here</a><br>
        The recommended backingboard construction is only applicable for <i>transportation or/and exhibition</i>.</li></div>
        """)
        WLG045_HTML = widgets.HTML(value = f"""
        <div style = 'margin: 0 0 0 20px;'><li><b>Caruso Isobond WLG045</b> &nbsp; more information: <a href="{WLG045_link}"
        target="_blank" style="color:blue; text-decoration:underline; font-weight:bold;">Click here</a><br>
        The recommended backingboard construction is only applicable for <i>transportation or/and exhibition</i>.<br></li></div>
        """)
        backboard_HTML = widgets.HTML(value = "The recommended backboard panel:")
        EBB8_HTML = widgets.HTML(value = f"""
        <div style = 'margin: 0 0 0 20px;'><li><b>Corrugated cardboards EBB 8.0 mm</b> &nbsp; more information: <a href="{EBB8_link}"
        target="_blank" style="color:blue; text-decoration:underline; font-weight:bold;">Click here</a></li></div>
        """)
        Honey7_HTML = widgets.HTML(value = f"""
        <div style = 'margin: 0 0 0 20px;'><li><b>Honeycomb cardboard 7.0 mm</b> &nbsp; more information: <a href="{Honey7_link}"
        target="_blank" style="color:blue; text-decoration:underline; font-weight:bold;">Click here</a></li></div>
        """)
        Honey13_HTML = widgets.HTML(value = f"""
        <div style = 'margin: 0 0 0 20px;'><li><b>Honeycomb cardboard 13.0 mm</b> &nbsp; more information: <a href="{Honey13_link}"
        target="_blank" style="color:blue; text-decoration:underline; font-weight:bold;">Click here</a></li></div>
        """)
        contact_HTML = widgets.HTML(value = """The specifications entered do not allow for assignment to a material we have tested for this application. Any use of this tool is at your own discretion.<br>
        Therefore, we recommend reviewing the cited publications in the <b>Disclaimer</b> tab before proceeding.<br><br>
        """)
        multi_HTML = widgets.HTML(value = "<div style = 'margin: 0 0 0 20px;'>Please select one of the recommended backboard panel above. The further down the list, the stiffer they are.</div>")

        # calculation Area A [m^2]
        A = None
        if canvas_shape == "circular":
            if canvas_values.get("diameter") not in (None, 0):
                A = math.pi*0.5*(canvas_values.get("diameter")/1000)**2
        elif canvas_shape == "oval":
            if None not in (canvas_values.get("width"), canvas_values.get("height")) and 0 not in (canvas_values.get("width"), canvas_values.get("height")):
                A = math.pi*(0.5*canvas_values.get("width")/1000)*(0.5*canvas_values.get("height")/1000)
        else:
            if None not in (canvas_values.get("width"), canvas_values.get("height")) and 0 not in (canvas_values.get("width"), canvas_values.get("height")):
                A = (canvas_values.get("width")/1000)*(canvas_values.get("height")/1000)

        # recommending the fiberfilling
        rec_fiberfilling = None
        if load_value != "restoration treatment":
            if A is None:
                rec_fiberfilling = widgets.HTML(value = "Please enter dimensions to get a recommendation for fiberfilling material.</div>")
            elif A > 1.5:
                rec_fiberfilling = contact_HTML
            elif deformation_value == "heavy":
                rec_fiberfilling = widgets.VBox([fiberfilling_HTML, WLG045_HTML])
            elif impasto_value in ("0%", "10%", "20%"):
                rec_fiberfilling = widgets.VBox([fiberfilling_HTML, WLG045_HTML])
            elif (impasto_value == "50%") or (deformation_value in ("not at all", "medium")):
                rec_fiberfilling = widgets.VBox([fiberfilling_HTML, WLG040_HTML])
            else:
                if A <= 0.3:
                    rec_fiberfilling = widgets.VBox([fiberfilling_HTML, WLG045_HTML]) if pretension_value in ("low", "low-medium") else widgets.VBox([fiberfilling_HTML, WLG040_HTML])
                elif 0.3 < A <= 0.45:
                    rec_fiberfilling = widgets.VBox([fiberfilling_HTML, WLG045_HTML]) if pretension_value in ("medium", "medium-high") else widgets.VBox([fiberfilling_HTML, WLG040_HTML])
                elif 0.45 < A <= 0.6:
                    rec_fiberfilling = widgets.VBox([fiberfilling_HTML, WLG045_HTML]) if pretension_value == "high" else widgets.VBox([fiberfilling_HTML, WLG040_HTML])
                elif 0.6 < A <= 1.5:
                    rec_fiberfilling = widgets.VBox([fiberfilling_HTML, WLG040_HTML])
        elif load_value == "restoration treatment":
            if A is None:
                rec_fiberfilling = widgets.HTML(value = "Please enter dimensions to get a recommendation for fiberfilling material.</div>")
            elif A > 1.5:
                rec_fiberfilling = contact_HTML
            else:
                rec_fiberfilling = widgets.VBox([fiberfilling_HTML, WLG035_HTML])
                
        #recommending the backingboard panel
        rec_backingboard = None
        if A is None:
            rec_backingboard = widgets.HTML(value = "Please enter dimensions to get a recommendation for backingboard panel.</div>")
        else:
            if A <= 0.35:
                rec_backingboard = widgets.VBox([backboard_HTML, EBB8_HTML, Honey7_HTML, Honey13_HTML, multi_HTML])
            elif 0.35 < A <= 0.8:
                rec_backingboard = widgets.VBox([backboard_HTML, Honey7_HTML, Honey13_HTML, multi_HTML])
            elif 0.8 < A <= 1.5:
                rec_backingboard = widgets.VBox([backboard_HTML, Honey13_HTML])
            else:
                rec_backingboard = contact_HTML
                
        # to show recommendation
        rec_display = []
        need_contact = False
        def is_contact(w):
            return (w is contact_HTML) or (hasattr(w, "value") and getattr(w, "value", "") == contact_HTML.value)
        for w in (rec_fiberfilling, rec_backingboard):
            if not w:
                continue
            if is_contact(w):
                need_contact = True
            else:
                rec_display.append(w)
        for w in rec_display:
            display(w)
        if need_contact:
            display(contact_HTML)
            
        # for download instruction
        display(download_box)
        
        # extract the values and show
        display(widgets.HTML("Summary of your specifications:"))
        preliminary_values = {
            "planned_mechanical_load": radiobutton_data["load"].value,
            "format_of_canvas": radiobutton_data["canvas"].value,
            "pretension": radiobutton_data["pretension"].value,
            "lining": radiobutton_data["lining"].value,
            "deformation": radiobutton_data["deformation"].value,
            "impasto": radiobutton_data["impasto"].value,
            "measurement": mea_radiobutton.value
        }

        # get the values
        canvas_unit_pairs, canvas_unit_map = active_units("canvas")
        stretcher_unit_pairs, stretcher_unit_map = active_units("stretcher")
        cwf_unit_pairs, cwf_unit_map = active_units("cwf")
        canvasstretcher_unit_pairs, canvasstretcher_unit_map = active_units("canvasstretcher")
        csd_unit_pairs, csd_unit_map = active_units("csd")

        # along to the choice of measurement mode
        mea_mode = (preliminary_values.get("measurement") or "").lower()
        if mea_mode.startswith("partially"):
            canvas_unit_pairs    = [p for p in canvas_unit_pairs    if p[0] != "weight"]
            stretcher_unit_pairs = [p for p in stretcher_unit_pairs if p[0] != "weight"]
            canvas_unit_map      = {k: v for k, v in canvas_unit_map.items() if k != "weight"}
            stretcher_unit_map   = {k: v for k, v in stretcher_unit_map.items() if k != "weight"}

        # read the values
        canvas_values = get_floattext_values("canvas", canvas_unit_pairs)
        stretcher_values = {
            "type_of_wood": radiobutton_data["stretcher"].value,
            **get_floattext_values("stretcher", stretcher_unit_pairs)
        }
        cwf_glazing_selected = radiobutton_data["cwf"].value
        cwf_values = {}
        if cwf_glazing_selected == "weight with glazing":
            cwf_values = {
                "glazing": cwf_glazing_selected,
                "type_of_glazing": radiobutton_data["glazing"].value,
                **get_floattext_values("cwf", cwf_unit_pairs)
            }
        elif cwf_glazing_selected == "weight without glazing":
            basic_cwf_pairs = [p for p in cwf_unit_pairs if not any(glazing_term in p[0] for glazing_term in ["glazing"])]
            cwf_values = {
                "glazing": cwf_glazing_selected,
                **get_floattext_values("cwf", basic_cwf_pairs)
            }
        else:
            cwf_values = {
                **get_floattext_values("cwf", cwf_unit_pairs)
            }
        canvasstretcher_values = get_floattext_values("canvasstretcher", canvasstretcher_unit_pairs)
        csd_values = get_floattext_values("csd", csd_unit_pairs)

        # display the results
        show_group("Preliminary:", preliminary_values)
        if mea_mode.startswith("individually"):
            show_group("Canvas:", canvas_values, canvas_unit_map)
            show_group("Stretcher:", stretcher_values, stretcher_unit_map)
            filtered_cwf_values = cwf_values.copy()
            if radiobutton_data["cwf"].value != "weight with glazing":
                glazing_keys = [k for k in filtered_cwf_values.keys() if "glazing" in k and k != "glazing"]
                for key in glazing_keys:
                    filtered_cwf_values.pop(key, None)
                if "type_of_glazing" in filtered_cwf_values:
                    filtered_cwf_values.pop("type_of_glazing", None)
            show_group("Decorative frame:", cwf_values, cwf_unit_map)
        elif mea_mode.startswith("partially"):
            show_group("Total (canvas + stretcher)", canvasstretcher_values, canvasstretcher_unit_map)
            show_group("Canvas:", canvas_values, canvas_unit_map)
            show_group("Stretcher:", stretcher_values, stretcher_unit_map)
            show_group("Decorative frame:", cwf_values, cwf_unit_map)
        elif mea_mode.startswith("as a whole"):
            show_group("Total (canvas + stretcher + decorative frame)", csd_values, csd_unit_map)
            show_group("Canvas:", canvas_values, canvas_unit_map)
            show_group("Stretcher:", stretcher_values, stretcher_unit_map)
            show_group("Decorative frame:", cwf_values, cwf_unit_map)

In [ ]:
# to show warning when the required values are not entered
display(HTML("""
<style>
    .red-input input {
        border: 2px solid red !important;
    }
</style>
""")
    )
## 4. calculation function
def f_calculation():
    with cal_output:
        clear_output()

        # common values for calculation and recommendation
        canvas_shape = radiobutton_data["canvas"].value
        canvas_unit_pairs, _ = active_units("canvas")
        canvas_values = get_floattext_values("canvas", canvas_unit_pairs)
        pretension_value = radiobutton_data["pretension"].value
        lining_value = radiobutton_data["lining"].value
        deformation_value = radiobutton_data["deformation"].value
        impasto_value = radiobutton_data["impasto"].value

        # show warning, in case the required values are not entered
        required_dimensions = ["diameter", "diameter of glazing"] if canvas_shape == "circular" else ["width", "height", "width of glazing", "height of glazing"]
        dimensions = ["width", "height", "diameter", "width of glazing", "height of glazing", "diameter of glazing"]
        missing_fields = []
        for key in dimensions:
            field = floattext_data.get(f"canvas_{key}")
            if not field:
                continue
            val = canvas_values.get(key)
            if key in required_dimensions:
                if val in (None, 0):
                    field.add_class("red-input")
                    missing_fields.append(key)
                else:
                    field.remove_class("red-input")
            else:
                field.remove_class("red-input")
                
        with warning_output:
            clear_output()
            if missing_fields:
                display(HTML(f"<div style='color:red'>Please enter all red-boxed values.</div>"))

        # initialization
        f = None
        
        # if yes:
        if f_toggle.value == True:
            if f_known.value is None:
                print("Please enter the first natural frequency.")
                return
            f = f_known.value
            
        # if no:
        else:
            # mapping factors through values user gave
            TensionFactors = {"low": 0.7, "low-medium": 0.8, "medium (ideal)": 1.0,
                                  "medium-high": 1.1, "high": 1.2}
            LiningFactors = {"0": 1, "1": 1.1, "2": 1.2, "3": 1.3}
            DeformFactors = {"not at all": 1, "medium": 1.1,
                                   "heavy": 1.2}
            ImpastoFactors = {"0%": 1, "10%": 1.1, "20%": 1.4,
                               "50%": 2}

            TensionFactor = TensionFactors.get(pretension_value, None)
            LiningFactor = LiningFactors.get(lining_value, None)
            DeformFactor = DeformFactors.get(deformation_value, None)
            ImpastoFactor = ImpastoFactors.get(impasto_value, None)

            # base value of parameters, later adjusted
            base_thickness_average = 2
            base_Emodule = 400
            base_density = 400

            # adjusting & sub-calculations
            if canvas_values["average thickness of painted canvas"] is not None and canvas_values["average thickness of painted canvas"] != 0:
                canvas_values["average thickness of painted canvas"] = canvas_values["average thickness of painted canvas"]
            else:
                if lining_value == "0":
                    canvas_values["average thickness of painted canvas"] = base_thickness_average
                else:
                    canvas_values["average thickness of painted canvas"] = base_thickness_average*2.5
            Emodule = base_Emodule*TensionFactor*LiningFactor*DeformFactor
            poisson = 0.3
            density = base_density*ImpastoFactor
            bending_stiffness = round((Emodule*canvas_values["average thickness of painted canvas"]**3)/(12*(1-poisson**2)), 2)

            # final result of calculation
            if canvas_shape == "circular":
                if canvas_values.get("diameter") in (None, 0):
                    f = None
                else:
                    f = round(((Emodule*canvas_values["average thickness of painted canvas"]**2)/
                              (12*(1-poisson**2)*density*(0.5*(canvas_values["diameter"]/1000))**4))**(1/4),
                              1)
            elif canvas_shape == "quadrangle":
                if canvas_values.get("width") in (None, 0) or canvas_values.get("height") in (None, 0):
                    f = None
                else:
                    f = round((math.pi/2)*
                              (math.sqrt(bending_stiffness/(density*canvas_values["average thickness of painted canvas"])))*
                              (1/(canvas_values.get("width")/1000)**2 + 1/(canvas_values.get("height")/1000)**2),
                              1)
            else:
                if canvas_values.get("width") in (None, 0) or canvas_values.get("height") in (None, 0):
                    f = None
                else:
                    diameter_oval = math.sqrt((canvas_values.get("width"))*(canvas_values.get("height")))
                    f_cir = ((Emodule*canvas_values["average thickness of painted canvas"]**2)/
                             (12*(1-poisson**2)*density*((0.5*diameter_oval/1000))**4))**(1/4)
                    f_qua = (math.pi/2)*(math.sqrt(bending_stiffness/(density*canvas_values["average thickness of painted canvas"])))*(1/(canvas_values.get("width")/1000)**2 + 1/(canvas_values.get("height")/1000)**2)
                    f = round(0.5*(f_cir + f_qua), 1)

            # to display the calculated frequency and according recommendation
            if f is not None:
                f_unknown_result = f"""
                <div style = 'margin: 20px 0 20px 0'>
                <b>Calculated first natural frequency:</b> {f: .1f} Hz
                </div>
                """
                display(HTML(f_unknown_result))
                update_result_state(f, 'calculated', preliminary_values.get('measurement'))
            else:
                f_unknown_result = f"""
                <div style = 'margin: 20px 0 20px 0'>
                <b>Calculated first natural frequency:</b> 
                <small>(e.g. 0.5 Hz)</small> 
                <i>wating for input...</i>
                </div>
                """
                display(HTML(f_unknown_result))
                result_summary(f=None, mode=None, mea_mode=None)

In [ ]:
## 6. update function
# for result
def update_result_state(f, mode, mea_mode):
    result_state["f"] = f
    result_state["mode"] = mode
    result_state["mea_mode"] = mea_mode
    result_summary(f, mode, mea_mode)
    
# to show right widgets after choosing toggle
def update_display(change = None):
    decision_output.clear_output()
    with decision_output:
        if f_toggle.value:
            f_calculation()
            display(f_known_box)
            if change and change.get("new") == True:
                result_state["f"] = None
                result_state["mode"] = None
                result_state["mea_mode"] = None
                result_summary(f=None, mode=None, mea_mode=None)
            else:
                result_summary(**result_state)
        else:
            display(f_unknown_box)
            f_calculation()
            if result_state["f"] is not None:
                result_summary(**result_state)

In [ ]:
## 7. for real-time according to the changes of value
# observe changes of toggle (yes <-> no)
f_toggle.observe(update_display, names='value')
# observe changes of given frequency
f_known.observe(
    lambda change: (
        f_calculation(), update_result_state(change['new'], 'entered', mea_radiobutton.value)
                   )
    if f_toggle.value and change['new'] and change['new'] > 0 else None,
    names = 'value'
)
# observe changes of parameters when the 'no' is selected
for key in ["pretension", "lining", "deformation", "impasto", "glazing"]:
     radiobutton_data[key].observe(lambda change: (
         f_calculation(), result_summary(result_state["f"], result_state["mode"], mea_radiobutton.value)),
         names = 'value')
for key_prefix in ["canvas", "stretcher", "cwf", "glazing", "canvasstretcher", "csd"]:
    for label, _ in dimension_unit_list:
        key = f"{key_prefix}_{label}"
        if key in floattext_data:
            floattext_data[key].observe(
               lambda change: (
                   f_calculation(), result_summary(result_state["f"], result_state["mode"], mea_radiobutton.value)),
                   names='value')
# observe changes of measurements of the artwork button
def _on_measurement_change(change):
    if change.get("name") != "value":
        return
    text = (change.get('new') or '').lower()
    if text.startswith('individually'):
        apply_measurement_mode("individually")
        _toggle_individual_weights(False)
    elif text.startswith('partially'):
        apply_measurement_mode("partially")
        _toggle_individual_weights(True)
    elif text.startswith('as a whole'):
        apply_measurement_mode("as a whole")
        _toggle_individual_weights(True)
    preliminary_values['measurement'] = (
        "individually" if text.startswith("individually")
        else "partially" if text.startswith("partially")
        else "as a whole" if text.startswith("as a whole")
        else None
    )
    result_summary(result_state["f"], result_state["mode"], mea_radiobutton.value)
mea_radiobutton.observe(_on_measurement_change, names = "value")
# observe changes of glazing
def on_cwf_glazing_change(change):
    glazing_value = change['new']
    if glazing_value == "weight with glazing":
        glazing_radiobutton.layout.display = 'block'
        canvas_shape = radiobutton_data["canvas"].value
        if canvas_shape == "circular":
            glazing_float_box_nor.layout.display = 'none'
            glazing_float_box_cir.layout.display = 'block'
        else:
            glazing_float_box_nor.layout.display = 'block'
            glazing_float_box_cir.layout.display = 'none'
    elif glazing_value == "weight without glazing":
        glazing_radiobutton.layout.display = 'none'
        glazing_float_box_nor.layout.display = 'none'
        glazing_float_box_cir.layout.display = 'none'
        radiobutton_data["glazing"].value = None
        glazing_fields = ["width of glazing", "height of glazing", "diameter of glazing", "thickness of glazing"]
        for field in glazing_fields:
            key = f"cwf_{field}"
            if key in floattext_data and floattext_data[key] is not None:
                floattext_data[key].value = ""
    else:
        glazing_radiobutton.layout.display = 'none'
        glazing_float_box_nor.layout.display = 'none'
        glazing_float_box_cir.layout.display = 'none'

radiobutton_data["cwf"].observe(on_cwf_glazing_change, names='value')

radiobutton_data["load"].observe(lambda change: (
    f_calculation(), result_summary(result_state["f"], result_state["mode"], mea_radiobutton.value)),
    names = 'value')
radiobutton_data["canvas"].observe(lambda change: (on_canvas_shape(change),
    f_calculation(), result_summary(result_state["f"], result_state["mode"], mea_radiobutton.value)),
    names = 'value')
radiobutton_data["stretcher"].observe(lambda change:
    result_summary(result_state["f"], result_state["mode"], mea_radiobutton.value), 
    names = 'value')
radiobutton_data["cwf"].observe(lambda change:
    result_summary(result_state["f"], result_state["mode"], mea_radiobutton.value), 
    names = 'value')
radiobutton_data["glazing"].observe(lambda change:
    result_summary(result_state["f"], result_state["mode"], mea_radiobutton.value), 
    names = 'value')

In [ ]:
update_display()

In [ ]:
## 8. display everything together as one flow
full_ui = widgets.Box([tabs], layout = widgets.Layout(width = "100%"))
display(full_ui)